In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [ ]:
class Data():
    def __init__(self, path):
        self.path = path
    
    def extract(self):
        df = pd.read_csv(self.path)
        return df

In [ ]:
data = Data('PMSM.csv')
df = data.extract()

In [ ]:
df.head(10)

In [ ]:
Ia = torch.tensor(df['Ia_amps'].to_numpy())
Ib = torch.tensor(df['Ib_amps'].to_numpy())
Ic = torch.tensor(df['Ic_amps'].to_numpy())
ang = torch.tensor(df['mAngle_rad'].to_numpy())
GaH = torch.tensor(df['GaH_V'].to_numpy())
GaL = torch.tensor(df['GaL_V'].to_numpy())
GbH = torch.tensor(df['GbH_V'].to_numpy())
GbL = torch.tensor(df['GbL_V'].to_numpy())
GcH = torch.tensor(df['GcH_V'].to_numpy())
GcL = torch.tensor(df['GcL_V'].to_numpy())

In [ ]:
l = len(Ia)

In [ ]:
input = []
output = []
for i in range(l):
    input.append([ang[i], GaH[i], GaL[i], GbH[i], GbL[i], GcH[i], GcL[i]])
    output.append([Ia[i], Ib[i], Ic[i]])

input = torch.tensor(input, dtype=torch.double)
output = torch.tensor(output, dtype=torch.double)

In [ ]:
input = input.to(torch.float32)
output = output.to(torch.float32)

predict control signals and angle, predict currents

In [ ]:
perm_indices = torch.randperm(input.size(0))
input = input[perm_indices]
perm_indices = torch.randperm(output.size(0))
output = output[perm_indices]
train_in = input[:1200001]
train_out = output[:1200001]
val_in = input[1200001:1600001]
val_out = output[1200001:1600001]
test_in = input[1600001:]
test_out = output[1600001:]

1 layer, 10 nodes

In [ ]:
class Motor(nn.Module):
    def __init__(self, train_in, train_out, val_in, val_out, test_in, test_out, criterion):
        super(Motor, self).__init__()
        self.train_in = train_in
        self.train_out = train_out
        self.val_in = val_in
        self.val_out = val_out
        self.test_in = test_in
        self.test_out = test_out
        self.criterion = criterion
    
    def model(self):
        model = nn.Sequential(
            nn.Linear(7, 60000),
            nn.Tanh(),
            nn.Linear(60000, 3),
            nn.ReLU()
        )
        return model
    
    def train(self, epochs, optimizer, model):
        train_loss = 0
        for epoch in range(epochs):
            preds = model(self.train_in)
            train_loss = torch.sum(self.criterion(preds, self.train_out))
            print(f'Epoch{epoch}: Train loss = {train_loss}')
            train_loss.backward()
            optimizer.step()
        return model
    
    def test(self, model):
        preds = model(self.test_in)
        test_loss = torch.sum(self.criterion(preds, self.test_out))
        print(f'Test loss = {test_loss}')

In [ ]:
criterion = F.mse_loss
motor = Motor(train_in, train_out, val_in, val_out, test_in, test_out, criterion)
model = motor.model()
optimizer = torch.optim.Adam(params=model.parameters(), lr = 0.01)
model = motor.train(100, optimizer, model)
motor.test(model)